# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [44]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

In [11]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    lem_tokens = []
    for t in tokens:
        lem_tokens.append(lemmatizer.lemmatize(t.lower()))
    return lem_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline =  Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier(class_weight='balanced'))
                     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=1)
pipeline.fit(X_tr, y_tr)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
pred = pipeline.predict(X_te)

In [17]:
for i, col in enumerate(y_te):
    print("Column: {}".format(col))
    print(classification_report(y_te[col], pred[:, i]))

Column: related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1550
          1       0.76      1.00      0.86      4951
          2       0.00      0.00      0.00        53

avg / total       0.57      0.76      0.65      6554

Column: request
             precision    recall  f1-score   support

          0       0.95      0.02      0.04      5415
          1       0.18      1.00      0.30      1139

avg / total       0.82      0.19      0.08      6554

Column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

Column: aid_related
             precision    recall  f1-score   support

          0       0.92      0.00      0.01      3815
          1       0.42      1.00      0.59      2739

avg / total       0.71      0.42      0.25      6554

Column: medical_help


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [40]:
y_tr['related']=y_tr['related'].map(lambda x: 1 if x == 2 else x) # To avoid exception


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
parameters = {'clf__n_estimators': [10, 20],
              'clf__min_samples_leaf': [2, 5]}
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)
cv.fit(X_tr, y_tr)



Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [10, 20], 'clf__min_samples_leaf': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
pred = cv.predict(X_te)
for i, col in enumerate(y_te):
    print("Column: {}".format(col))
    print(classification_report(y_te[col], pred[:, i]))

Column: related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1550
          1       0.76      1.00      0.86      4951
          2       0.00      0.00      0.00        53

avg / total       0.57      0.76      0.65      6554

Column: request
             precision    recall  f1-score   support

          0       0.93      0.68      0.78      5415
          1       0.33      0.76      0.46      1139

avg / total       0.83      0.69      0.73      6554

Column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

Column: aid_related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      3815
          1       0.42      1.00      0.59      2739

avg / total       0.17      0.42      0.25      6554

Column: medical_help


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))])

parameters_2 = {'clf__estimator__min_samples_split': [3, 5, 8],
                  'clf__estimator__max_depth': [3, 7, 10]}

cv_2 = GridSearchCV(estimator=pipeline_2, param_grid=parameters_2, verbose=1)
cv_2.fit(X_tr, y_tr)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


### 9. Export your model as a pickle file

In [ ]:
with open('optimized_cv.pkl', 'wb') as f:
    pickle.dump(cv_2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.